<a href="https://colab.research.google.com/github/sakshamsonker12/Myfirstrepo/blob/main/NER_SPACY.IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
! pip install -U spacy -q 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.5 MB/s eta 0:00:00


In [14]:
!python -m spacy info

2023-06-11 19:11:22.282889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.3                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.107+-x86_64-with-glibc2.31
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.5.0)        



In [15]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [16]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)
     

In [17]:
TRAIN_DATA

{'classes': ['CRYPTO', 'VALUE', 'PERCENTAGE', 'ORG', 'PERSON'],
 'annotations': [['Cryptocurrency prices today surged with Bitcoin trading at $48,089.82, a 2.6% increase in the last 24 hours. Ethereum surged marginally to $3,838.45, a 1.3% increase in the last 24 hours.',
   {'entities': [[40, 47, 'CRYPTO'],
     [59, 69, 'VALUE'],
     [73, 77, 'PERCENTAGE'],
     [109, 117, 'CRYPTO'],
     [139, 148, 'VALUE'],
     [152, 156, 'PERCENTAGE']]}],
  ['Dogecoin went up 15.5% at $0.181994 in the last 24 hours, after Tesla Inc chief Elon Musk said on Tuesday the electric carmaker will accept Dogecoin as payment for merchandise on a test basis, sending the meme-based cryptocurrency up over 20%. Dogecoin, popular among retail investors, raced up to $0.20 after the tweet.',
   {'entities': [[0, 8, 'CRYPTO'],
     [64, 69, 'ORG'],
     [80, 89, 'PERSON'],
     [140, 148, 'CRYPTO'],
     [239, 242, 'PERCENTAGE'],
     [298, 303, 'VALUE']]}],
  ['Shiba Inu witnessed a marginal surge to $0.0000335

In [18]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 5/5 [00:00<00:00, 368.51it/s]


In [20]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2023-06-11 19:17:10.879855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-06-11 19:17:45.799144: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-06-11 19:17:48,583] [INFO] Set up nlp object from config
[2023-06-11 19:17:48,605] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-11 19:17:48,610] [INFO] Created vocabulary
[2023-06-11 19:17:48,610] [INFO] Finished initializing nlp object
[2023-06-11 19:17:48,770] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.05    3.70    2.78    5.56    0.04
 78     200        443.16   1255.92  100.00  100.00  100.00 

In [25]:
nlp_ner = spacy.load("/content/model-best") 


In [23]:
doc = nlp_ner('''s of Saturday, June 10, bitcoin (BTC) is still the undisputed leader with a market valuation of $496 billion, accounting for 47.5% of the entire crypto economy’s $1.04 trillion market capitalization. However, BTC has experienced a slight dip of 3.32% in the past day and 5.79% over the week against the U.S. dollar, according to current data. Interestingly, while BTC has weathered the storm relatively well, many other top crypto assets, particularly those in the top 20, have taken a much harder hit in terms of value loss.

Bitcoin Dominance Soars to 47.5%: Highest Level in a Year as Other Crypto Assets Suffer
Bitcoin (BTC) dominance according to coinmarketcap.com on Saturday, June 10, 2023.
The top 20 crypto assets have seen some major losses this week, with a handful of them experiencing double-digit dips. BNB, ADA, TRX, SOL, MATIC, DOGE, LTC, DOT, AVAX, SHIB, and ATOM are among the biggest losers. In comparison, ethereum (ETH), the second leading crypto asset, only lost 8.29%. While BTC did experience a brief surge in dominance to 47.18% on June 5, its current dominance rating is even higher at 47.5%. Interestingly, the last time BTC’s dominance was this high was almost exactly a year ago on June 11, 2022, when it was hovering at around 47.49%.

While BTC is currently dominating the crypto world, ethereum has managed to hold its own with a 20% dominance rating and a market capitalization of $208 billion. Meanwhile, the stablecoin tether (USDT) is still going strong with a market valuation of $83.39 billion, accounting for 7.96% of the total $1.04 trillion in value. ''') # input sample text
     

In [24]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter